In [8]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
%matplotlib inline

In [3]:
framingham = pd.read_csv("/Users/zoetsai/Downloads/framingham.csv")

In [5]:
def dfStr(df):
    print("The dataframe contains {0} rows and {1} columns".format(df.shape[0], df.shape[1]))
    print("The data types of columns are: n")
    print(df.dtypes)

In [6]:
dfStr(framingham)

The dataframe contains 4240 rows and 16 columns
The data types of columns are: n
male                 int64
age                  int64
education          float64
currentSmoker        int64
cigsPerDay         float64
BPMeds             float64
prevalentStroke      int64
prevalentHyp         int64
diabetes             int64
totChol            float64
sysBP              float64
diaBP              float64
BMI                float64
heartRate          float64
glucose            float64
TenYearCHD           int64
dtype: object


In [9]:
from sklearn.cross_validation import train_test_split
train0, test0 = train_test_split(framingham.loc[framingham['TenYearCHD'] == 0,:], train_size=0.65, random_state=1000)
train1, test1 = train_test_split(framingham.loc[framingham['TenYearCHD'] == 1,:], train_size=0.65, random_state=1000)
framinghamTrain = pd.DataFrame(np.vstack((train0,train1)), columns=framingham.columns).convert_objects(convert_numeric=True)
framinghamTest = pd.DataFrame(np.vstack((test0,test1)), columns=framingham.columns).convert_objects(convert_numeric=True)

/Users/zoetsai/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/zoetsai/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [12]:
predictors = framinghamTrain.columns.tolist()
predictors.remove('TenYearCHD')
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(framinghamTrain[predictors])
X = imp.transform(framinghamTrain[predictors])
y = framinghamTrain['TenYearCHD']

In [13]:
from sklearn.linear_model import LogisticRegression
framinghamLog = LogisticRegression(penalty='l1', tol=1e-6)
framinghamLog.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False)

In [14]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(framinghamTest[predictors])
Xtest = imp.transform(framinghamTest[predictors])
ytest = framinghamTest['TenYearCHD']
predictTest = framinghamLog.predict(Xtest)

In [16]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(ytest==1, predictTest >= 0.5))

[[1249   10]
 [ 208   18]]


In [23]:
for t in zip(predictors, framinghamLog.coef_.tolist()[0]):
    print(t)

('male', 0.3683800172151044)
('age', 0.06251021786713819)
('education', -0.052718680607569714)
('currentSmoker', 0.031775051434933324)
('cigsPerDay', 0.020673472278317385)
('BPMeds', 0.21176995814005264)
('prevalentStroke', 0.24139462253618732)
('prevalentHyp', 0.33152560729751085)
('diabetes', 0.043853193756420154)
('totChol', 0.0007209523880071466)
('sysBP', 0.014670174621243525)
('diaBP', -0.005872589436499207)
('BMI', 0.0002778333926823566)
('heartRate', -0.005754989325194048)
('glucose', 0.009823503517720412)
